In [1]:
# imports
from bs4 import BeautifulSoup
from pandas import DataFrame

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from time import sleep
from json import load

In [2]:
# laoding the companpy names.
with open('./nifty_500_symbols.json','rt') as json_file:
    nifty_500 = load(json_file)

In [ ]:
# nifty_500.keys()

In [3]:
# starting a browser session
options = webdriver.EdgeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
browser = webdriver.Edge(options=options)

browser.get('https://trendlyne.com/features/')

In [4]:
# first load
# close popups
popup = browser.find_element(By.CLASS_NAME, value='close')
popup.click()

In [5]:
def get_table_markup(commany_symbol):
    # select fundamentals from dropdown
    dropdown = browser.find_element(By.CLASS_NAME, value='input-group-btn')
    dropdown.click()
    fundamentals_link = browser.find_element(By.LINK_TEXT, 'Fundamentals')
    fundamentals_link.click()

    # search the scrips for fundamentals
    search_box = browser.find_element(By.NAME,value='search')
    search_box.clear()
    search_box.click()
    search_box.send_keys(commany_symbol)
    sleep(2)
    
    #NEW CODE
    search_box.send_keys(Keys.ARROW_DOWN)
    matches = browser.find_element(By.CLASS_NAME,value='ui-menu')
    
    search_box.send_keys(Keys.ENTER)
    
    # after being on fundamentals page
    sleep(3)
    table = browser.find_element(By.CLASS_NAME,value='rt-table')
    table_html = table.get_attribute('outerHTML')
    return table_html

def markup_to_df(markup,company_name):
    soup = BeautifulSoup(markup=markup,features='lxml')
    # scraping table headers
    headers = soup.find_all('span', {'class': 'text-xs-center'})
    cols = [header.text for header in headers]
    cols.remove('Graph')
    
    # scraping actual table data.
    rows = []
    new_row = []
    spans = soup.find_all('div', {'class': 'rt-td'})
    for span in spans:
        idx = span.text
        striped_idx = idx.strip()
        if striped_idx:
            if striped_idx[0].isalpha():
                rows.append(new_row)
                new_row = [striped_idx]
            else:
                striped_idx_rem_comma = striped_idx.replace(',', '')
                new_row.append(striped_idx_rem_comma)
            if striped_idx == 'Net profit TTM Cr':
                break

    # skipping first row being empty.
    table_data = rows[1:]

    # merging headers and data.
    df = DataFrame(table_data, columns=cols)

    # saving to csv file.
    df.to_csv(f'./nifty_500_scraped_data_raw/{company_name}.csv',index=False)

In [7]:
# sample_companies = {"Adani Ports and Special Economic Zone Ltd.": "ADANIPORTS",
#                     "Aditya Birla Capital Ltd.": "ABCAPITAL",
#                     "Aditya Birla Fashion and Retail Ltd.": "ABFRL",
#                     "Aditya Birla Sun Life AMC Ltd.": "ABSLAMC",
#                     "Asian Paints Ltd.": "ASIANPAINT", }
# sample_companies = {"Trident Ltd.": "TRIDENT LTD",
#                     "Container Corporation of India Ltd.": "CONCOR CORP", }


In [10]:
try:
    for company_name,company_symbol in nifty_500.items():
        table_markup = get_table_markup(company_symbol)
        markup_to_df(table_markup,company_name)
except Exception as e:
    print(e)